In [ ]:
import pyaudio

p = pyaudio.PyAudio()
info = p.get_host_api_info_by_index(0)
num_devices = info.get('deviceCount')

print("Available audio input devices:")
for i in range(num_devices):
    device_info = p.get_device_info_by_host_api_device_index(0, i)
    if device_info.get('maxInputChannels') > 0:
        print(f"Device {i}: {device_info.get('name')}")


Method 1: recording sound and passing it as a .wav file to the model


Part 1 : recording the sound

In [ ]:
import pyaudio
import wave

CHUNK = 512
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 48000
RECORD_SECONDS = 10
WAVE_OUTPUT_FILENAME = "output.wav"

p = pyaudio.PyAudio()

# print the available devices
info = p.get_host_api_info_by_index(0)
num_devices = info.get('deviceCount')
for i in range(num_devices):
    if (p.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
        print(f"Input Device id {i} - {p.get_device_info_by_host_api_device_index(0, i).get('name')}")

device_id = int(input("Select the device id: "))

print(f"Recording from {p.get_device_info_by_host_api_device_index(0, device_id).get('name')} for {RECORD_SECONDS} seconds...")

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                input_device_index=device_id,
                frames_per_buffer=CHUNK)

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("Recording complete.")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

print(f"Exported audio as {WAVE_OUTPUT_FILENAME}")


Method 2 (ADVANCED) passing sound and dealing with it in real-time

In [ ]:
import sounddevice as sd
import numpy as np

# Set parameters for recording
duration = 60  # seconds
fs = 48000  # sampling rate
device = 6  # device id for recording (id 6 is for the current testing settings)
global sp
sp = False #boolean to keep in check the number of times a surpass of amplitude is detected

# Define function to print "passed" when amplitude goes over a certain level
def audio_callback(indata, frames, time, status):
    global sp
    amplitude = np.abs(indata).max()
    if amplitude > 0.25 and sp == False:
        sp = True
        print("passed")
    elif amplitude < 0.25 and sp == True:
        sp = False
    return None

# Start recording
print("Recording...")
with sd.InputStream(channels=1, samplerate=fs, device=device, callback=audio_callback):
    sd.sleep(int(duration * 1000))
